[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.0MB 5.8MB/s 
     |████████████████████████████████| 3.2MB 25.0MB/s 
     |████████████████████████████████| 870kB 38.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=268374c2d924295edbeaf836a44c300a640d08d02949b3eee23e919ecd3bc680
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
from google.colab import drive

current_directory = '/content/drive/My Drive/FSem/'
drive.mount('/content/drive')

Mounted at /content/drive


# Headers and Global Variables

In [ ]:
import csv
import torch
from torch.nn.functional import softmax
from torch.nn.functional import relu
from transformers import BertForNextSentencePrediction, BertTokenizer
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
from IPython.display import display, HTML

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking')

# Datasets

In [ ]:
class ClozeTest(torch.utils.data.Dataset):
    def __init__(self, filename):
        
        dataset = []

        with open(current_directory + filename, 'r', encoding='utf-8') as d:
            reader = csv.reader(d, quotechar='"', delimiter=',', 
                                quoting=csv.QUOTE_ALL, skipinitialspace=True)                
            for line in reader:
                dataset.append(line)
            dataset.pop(0)

        self.data = []
        self.labels = []

        for sample in dataset:
            
            start = " ".join(sample[1:-3])
            end1 = sample[-3]
            end2 = sample[-2]
            right_ending = sample[-1]

            self.data.append([start, end1])
            self.labels.append(0 if "1" == right_ending else 1)

            self.data.append([start, end2])
            self.labels.append(0 if "2" == right_ending else 1)

    def __getitem__(self, idx):
        X = self.data[idx]
        y = self.labels[idx]        
        return X, y

    def __len__(self):
        assert len(self.data) == len(self.labels)
        return len(self.labels)

In [ ]:
triggers_only_set = ClozeTest('cloze_test_triggers_only.csv')
triggers_removed_only_set = ClozeTest('cloze_test_triggers_removed_only.csv')
triggers_synonymized_only_set = ClozeTest('cloze_test_triggers_synonymized_only.csv')

# Saliency Map

In [ ]:
def saliency_map(model, tokenizer, input, ending, label, first_row):
    # Activations are saved.
    acts = dict() # one-key dictionary. Doesn't work otherwise.
    def get_acts(name):
        def hook(module, input, output):
            acts[name] = output.detach()
        return hook

    # Gradients are saved.
    grads = dict() # same as for activations
    def get_grads(name):
        def hook(module, input, output):
            grads[name] = output[0].detach() # 'output' is a tuple
        return hook

    frw_handle = model.bert.embeddings.register_forward_hook(get_acts("emb"))
    bck_handle = model.bert.embeddings.register_backward_hook(get_grads("emb"))

    tokens = tokenizer(input, text_pair=ending, return_tensors='pt')
    token_names = tokenizer.tokenize(tokenizer.decode(tokens.input_ids[0]))

    model.eval()
    model.zero_grad()

    logits = model(**tokens, labels=torch.tensor([label])).logits.view(-1, 2)
    prediction = logits.argmax(dim=-1)
    # Gradient of loss as per Han et al. 2020 calculated
    torch.nn.CrossEntropyLoss()(logits, prediction).backward()

    frw_handle.remove()
    bck_handle.remove()

    saliencies = (-grads["emb"] * acts["emb"]).sum(dim=-1)
    norm = torch.linalg.norm(saliencies, ord=1, dim=-1, keepdims=True)
    saliencies = saliencies / norm # normalizing the saliencies
    saliencies = saliencies[0] # squeezing the batch of one

    # Visualization. Courtesy of https://gist.github.com/ihsgnef
    colors = saliencies / max(abs(saliencies.min()), abs(saliencies.max())) * 0.5 + 0.5
    cmap = matplotlib.cm.get_cmap('RdBu')
    template = '<span class="barcode"; style="color: black; background-color: {}">{}</span>'
    colored_string = ''
    for token_name, color in zip(token_names, colors.tolist()):
        color = matplotlib.colors.rgb2hex(cmap(color)[:3])
        colored_string += template.format(color, '&nbsp' + token_name + '&nbsp')
    
    colored_string = first_row + "\tlabel:" + str(label) + "\tprediction:" + str(prediction[0].item()) + "\t" + colored_string
    display(HTML(colored_string))

# Differences visualizer

In [ ]:
examples = {'instead': 0, 'ever': 17, 'anymore': 26, 'too': 8, 'eventually': 30, 'immediately': 42, 'anyway': 5, 'soon': 25, 'later': 38, 'now': 20, 'finally': 3}

In [ ]:
def visualize_for_model(model):
    for trigger, sentence_id in examples.items():
        print("sentences with " + trigger + ":")
        story, label = triggers_only_set[sentence_id]
        saliency_map(model, tokenizer, story[0], story[1], label, "with triggers")
        story, label = triggers_removed_only_set[sentence_id]
        saliency_map(model, tokenizer, story[0], story[1], label, "triggers removed")
        story, label = triggers_synonymized_only_set[sentence_id]
        saliency_map(model, tokenizer, story[0], story[1], label, "triggers synonymized")
        print()

# Cloze Only

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(current_directory + 'shared/bertfornsp_clozeonly_finetuned10')
visualize_for_model(model)

sentences with instead:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



sentences with ever:



sentences with anymore:



sentences with too:


...


sentences with anyway:



sentences with soon:



sentences with later:



sentences with now:



sentences with finally:


# Roc Only

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(current_directory + 'shared/bertfornsp_roc_finetuned1')
visualize_for_model(model)

sentences with instead:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



sentences with ever:



sentences with anymore:



sentences with too:


...


sentences with anyway:



sentences with soon:



sentences with later:



sentences with now:



sentences with finally:


# 1. Roc 2. Cloze

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(current_directory + 'shared/bertfornsp_cloze_finetuned10')
visualize_for_model(model)

sentences with instead:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



sentences with ever:



sentences with anymore:



sentences with too:


...


sentences with anyway:



sentences with soon:



sentences with later:



sentences with now:



sentences with finally:


# 1. Cloze 2. Roc

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(current_directory + 'alternative/trained_model')
visualize_for_model(model)

sentences with instead:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



sentences with ever:



sentences with anymore:



sentences with too:


...


sentences with anyway:



sentences with soon:



sentences with later:



sentences with now:



sentences with finally:


# 1. Roc 2. Cloze with Squared Error Loss, Hypothesis Only data with target probability (0.5, 0.5)

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(current_directory + 'shared/bertfornsp_cloze_finetuned_bias_reduced10')
visualize_for_model(model)

sentences with instead:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



sentences with ever:



sentences with anymore:



sentences with too:


...


sentences with anyway:



sentences with soon:



sentences with later:



sentences with now:



sentences with finally:


# Cloze with 5 000 Roc Stories mixed in

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(current_directory + 'shared/bertfornsp_mixed5')
visualize_for_model(model)

sentences with instead:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



sentences with ever:



sentences with anymore:



sentences with too:


...


sentences with anyway:



sentences with soon:



sentences with later:



sentences with now:



sentences with finally:


# Cloze with 10 000 Roc Stories mixed in

In [ ]:
model = BertForNextSentencePrediction.from_pretrained(current_directory + 'shared/bertfornsp_mixed_more_roc5')
visualize_for_model(model)

sentences with instead:


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "



sentences with ever:



sentences with anymore:



sentences with too:


...


sentences with anyway:



sentences with soon:



sentences with later:



sentences with now:



sentences with finally:
